In [79]:
import os, glob, sqlite3
import pandas
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer


In [80]:
os.makedirs("Sleep and Mood Dataset", exist_ok=True)
Dataset_Path = os.path.join("Sleep and Mood Dataset", "Sleep and Mood Dataset.csv")

In [81]:
Dataset = pandas.read_csv(Dataset_Path)

In [82]:
class Dataset_Cleanning:
    def __init__(self, Dataset):
        self.Dataset = Dataset.copy().dropna()
        self.Cleaned_Dataset = self.Data_cleaning()
        
    def Data_cleaning(self):
        for Column_Name in self.Dataset.columns:
            if not pandas.api.types.is_numeric_dtype(self.Dataset[Column_Name]):
                pass
                # Column_Cleaning = {Values: Index for Index, Values in enumerate(self.Dataset[Column_Name].unique())}
                # self.Dataset.loc[:, Column_Name] = self.Dataset[Column_Name].map(Column_Cleaning)    
        
        self.Cleaned_Dataset = self.Dataset
        return self.Cleaned_Dataset

In [83]:
class SQLite3:
    def __init__(self, Dataset , Name):
        self.Dataset = Dataset
        self.Name = Name
        
    def SQLite3_Connection(self):
        if self.Name + ".db" not in glob.glob("*db"):
            Connection = sqlite3.connect(self.Name + ".db")
            self.Dataset.to_sql(self.Name, Connection, if_exists='replace', index=False)
            print("Database already created!")
            return Connection
        else: return sqlite3.connect(self.Name + ".db")

    def Exc(self, Query):
        Connection = self.SQLite3_Connection()
        return pandas.read_sql_query(Query, Connection)

Table_Name = 'Sleep and Mood'
Conn = SQLite3(Dataset_Cleanning(Dataset).Cleaned_Dataset, Table_Name)
Query = f"""
    SELECT * 
    FROM '{Table_Name}'
"""        
Conn.Exc(Query)

,Employee ID,Employment Type,Hours Worked Per Week,Productivity Score,Well Being Score
0,1,Remote,29,75,78
1,2,In-Office,45,49,47
2,3,Remote,34,74,89
3,4,Remote,25,81,84
4,5,Remote,50,70,74
...,...,...,...,...,...
995,996,Remote,33,88,82
996,997,Remote,33,88,73
997,998,In-Office,45,74,61
998,999,In-Office,57,50,52


In [89]:
Data_For_Training = Dataset.copy().dropna()

X = Data_For_Training[['Employment Type', 'Hours Worked Per Week', 'Productivity Score']]
Y = Data_For_Training['Well Being Score']

Preprocessor = ColumnTransformer(
    transformers=[
        ('Data Type Conversion', StandardScaler(), ['Hours Worked Per Week', 'Productivity Score']),
        ('Categorical Encoding', OneHotEncoder(), ['Employment Type'])
    ])      
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)


X_train = Preprocessor.fit_transform(X_train)
X_test = Preprocessor.transform(X_test)

model = tf.keras.Sequential([
    tf.keras.layers.Dense(128, activation='relu', input_shape=(X_train.shape[1],)),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(1)
])

model.compile(optimizer='adam', loss='mse', metrics=['mae'])

History = model.fit(X_train, Y_train, validation_data=(X_test, Y_test), epochs=1000, batch_size=32)


Epoch 1/1000
25/25 [==============================] - 0s 7ms/step - loss: 4185.4482 - mae: 63.2052 - val_loss: 3956.6384 - val_mae: 61.3721
Epoch 2/1000
25/25 [==============================] - 0s 3ms/step - loss: 3556.8992 - mae: 57.9788 - val_loss: 2791.1733 - val_mae: 50.9617
Epoch 3/1000
25/25 [==============================] - 0s 4ms/step - loss: 1650.7942 - mae: 37.1748 - val_loss: 525.7785 - val_mae: 19.2682
Epoch 4/1000
25/25 [==============================] - 0s 4ms/step - loss: 327.3459 - mae: 14.2636 - val_loss: 281.1371 - val_mae: 13.2811
Epoch 5/1000
25/25 [==============================] - 0s 4ms/step - loss: 244.1202 - mae: 12.3513 - val_loss: 246.2382 - val_mae: 12.6461
Epoch 6/1000
25/25 [==============================] - 0s 3ms/step - loss: 219.3286 - mae: 11.7203 - val_loss: 223.2023 - val_mae: 11.9096
Epoch 7/1000
25/25 [==============================] - 0s 3ms/step - loss: 203.4368 - mae: 11.2498 - val_loss: 211.3018 - val_mae: 11.4778
Epoch 8/1000
25/25 [=========

In [93]:
New_Data = pandas.DataFrame({
    'Employment Type': ['Remote'], 
    'Hours Worked Per Week': [87], 
    'Productivity Score': [52]
})

New_Data_Processed = Preprocessor.transform(New_Data)

predicted_price = model.predict(New_Data_Processed)
print(f'Well Being Score: {predicted_price[0][0]:,.2f}')

1/1 [==============================] - 0s 20ms/step
Well Being Score: 94.45
